# Konverze modelu do nasaditelného formátu (.tflite → C++)

Tento krok převádí model TensorFlow do formátu .tflite a následně do C pole, které lze připojit k firmware.

In [1]:
import tensorflow as tf
import numpy as np

# === 1. Vytvoření a trénink jednoduchého modelu ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(10,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy')
X_train = np.random.rand(100, 10).astype(np.float32)
y_train = np.random.randint(0, 2, size=(100, 1)).astype(np.float32)
model.fit(X_train, y_train, epochs=5, verbose=0)

# === 2. Konverze modelu do TFLite formátu ===
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# === 3. Uložení modelu .tflite ===
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

# === 4. Převedení do C pole (pro přiložení do firmwaru)
# Spuštění v terminálu (ručně):
# xxd -i model.tflite > model_data.cc


2025-04-08 20:54:31.466784: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


INFO:tensorflow:Assets written to: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmp5dy257x6/assets


2025-04-08 20:54:31.883376: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2025-04-08 20:54:31.883387: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2025-04-08 20:54:31.883940: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmp5dy257x6
2025-04-08 20:54:31.884537: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-04-08 20:54:31.884542: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmp5dy257x6
2025-04-08 20:54:31.885551: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2025-04-08 20:54:31.885793: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2025-04-08 20:54:31.898669: I tensorflow/cc/saved_model/loader.

# Integrace modelu do aplikačního kódu (Python simulace)

V embedded prostředí to probíhá v C/C++, ale zde si to ukážeme v Pythonu jako simulaci:

In [2]:
# === Inference simulace (na embedded zařízení) ===
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Simulace vstupních dat (např. ze senzoru)
input_data = np.random.rand(1, 10).astype(np.float32)

# Předání vstupu a spuštění modelu
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Výstup
output = interpreter.get_tensor(output_details[0]['index'])
print(f"Predikce modelu: {output[0][0]:.4f}")

Predikce modelu: 0.5392


INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
